<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

analyse de régression du football européen  <br><br> avec scikit-learn

<br><br></p>


Nous allons utiliser une autre fois un dataset open issue du <a href="https://www.kaggle.com">Kaggle</a> 

ce dataset <a href="https://www.kaggle.com/hugomathien/soccer">European Soccer Database</a> contient plus de 25,000 matches et plus de 10,000  joueurs professionnels du football européen (saisons entre 2008 et 2016).


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importer les bibliothèques<br><br></p>


In [1]:
import sqlite3
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>
lire les données de la base de données vers Pandas 
<br><br></p>


In [2]:
# Creer une connexion .
cnx = sqlite3.connect('soccer/database.sqlite')
#df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [3]:
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [4]:
df.head()

id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00            67.0   
1   2              218353         505942  2015-11-19 00:00:00            67.0   
2   3              218353         505942  2015-09-21 00:00:00            62.0   
3   4              218353         505942  2015-03-20 00:00:00            61.0   
4   5              218353         505942  2007-02-22 00:00:00            61.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium              medium      49.0   
1       71.0          right              medium              medium      49.0   
2       66.0          right              medium              medium      49.0   
3       65.0          right              medium              medium      48.0   
4       65.0          right              medium              medium      48.0   

      ...       vision  penalties  marking  standing_tackle  sliding_tackle  \
0     ...         54.0       48.0     65.0             69.0            69.0   
1     ...         54.0       48.0     65.0             69.0            69.0   
2     ...         54.0       48.0     65.0             66.0            69.0   
3     ...         53.0       47.0     62.0             63.0            66.0   
4     ...         53.0       47.0     62.0             63.0            66.0   

   gk_diving  gk_handling  gk_kicking  gk_positioning  gk_reflexes  
0        6.0         11.0        10.0             8.0          8.0  
1        6.0         11.0        10.0             8.0          8.0  
2        6.0         11.0        10.0             8.0          8.0  
3        5.0         10.0         9.0             7.0          7.0  
4        5.0         10.0         9.0             7.0          7.0  

[5 rows x 42 columns]

In [5]:
df.shape

(183978, 42)

In [6]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Déclarons les colonnes à utiliser comme caractéristiques 
<br><br></p>


In [7]:
features = [
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Specifier une cible de prédiction 
<br><br></p>


In [8]:
target = ['overall_rating']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Nettoyer les données <br><br></p>


In [9]:
df = df.dropna()

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Extraire les caractéristiques et les valeurs de la cible ('overall_rating') vers un Dataframes séparé 
<br><br></p>


In [10]:
X = df[features]

In [11]:
y = df[target]

affichons une ligne de nos caractéristiques : 

In [12]:
X.iloc[2]

potential             66.0
crossing              49.0
finishing             44.0
heading_accuracy      71.0
short_passing         61.0
volleys               44.0
dribbling             51.0
curve                 45.0
free_kick_accuracy    39.0
long_passing          64.0
ball_control          49.0
acceleration          60.0
sprint_speed          64.0
agility               59.0
reactions             47.0
balance               65.0
shot_power            55.0
jumping               58.0
stamina               54.0
strength              76.0
long_shots            35.0
aggression            63.0
interceptions         41.0
positioning           45.0
vision                54.0
penalties             48.0
marking               65.0
standing_tackle       66.0
sliding_tackle        69.0
gk_diving              6.0
gk_handling           11.0
gk_kicking            10.0
gk_positioning         8.0
gk_reflexes            8.0
Name: 2, dtype: float64

affichons notre cible : 

In [21]:
y

overall_rating
0                 67.0
1                 67.0
2                 62.0
3                 61.0
4                 61.0
5                 74.0
6                 74.0
7                 73.0
8                 73.0
9                 73.0
10                73.0
11                74.0
12                73.0
13                71.0
14                71.0
15                71.0
16                70.0
17                70.0
18                70.0
19                70.0
20                70.0
21                70.0
22                69.0
23                69.0
24                69.0
25                69.0
26                69.0
27                69.0
28                69.0
29                68.0
...                ...
183933            76.0
183934            75.0
183935            77.0
183936            77.0
183937            63.0
183938            63.0
183939            63.0
183940            63.0
183941            63.0
183942            66.0
183943            66.0
183944            66.0
183945            66.0
183946            66.0
183947            68.0
183948            68.0
183949            68.0
183950            68.0
183951            67.0
183952            67.0
183968            78.0
183969            81.0
183970            81.0
183971            81.0
183972            83.0
183973            83.0
183974            78.0
183975            77.0
183976            78.0
183977            80.0

[180354 rows x 1 columns]

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>
Holdout de notre dataset 
<br><br></p>


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(1) la régression linéaire: adapter le modèle aux données d'apprentissage 
<br><br></p>


In [16]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Effectuer la prédiction en utilisant un modèle de régression linéaire
<br><br></p>


In [17]:
y_prediction = regressor.predict(X_test)
y_prediction

array([[66.51284879],
       [79.77234615],
       [66.57371825],
       ...,
       [69.23780133],
       [64.58351696],
       [73.6881185 ]])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

C'est quoi la moyenne des valeurs attendues de la cible dans l'ensemble de test ?
<br><br></p>


In [18]:
y_test.describe()

overall_rating
count    59517.000000
mean        68.635818
std          7.041297
min         33.000000
25%         64.000000
50%         69.000000
75%         73.000000
max         94.000000

mean = 68.635818

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluer la peformance de la régression en utilisant  Root Mean Square Error

<br><br></p>


In [19]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [22]:
print(RMSE)

2.805303046855224


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(2) la régression en utilisant les arbres de décsion: adapter un nouveau modèle de régression aux données de test 
<br><br></p>


In [23]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

effectuer la régression en utilisant les arbres de décision 
<br><br></p>


In [24]:
y_prediction = regressor.predict(X_test)
y_prediction

array([62.        , 84.        , 62.38666667, ..., 71.        ,
       62.        , 72.        ])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

comparaison: C'est quoi la moyenne des valeurs attendues de la cible dans l'ensemble de test ?
<br><br></p>


In [25]:
y_test.describe()

overall_rating
count    59517.000000
mean        68.635818
std          7.041297
min         33.000000
25%         64.000000
50%         69.000000
75%         73.000000
max         94.000000

mean = 68.635818

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold">

Evaluer la peformance de la régression (en utilisant les arbres de décison) en utilisant  Root Mean Square Erro </p>

In [26]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [27]:
print(RMSE)

1.457761451571687


## KNN Regression

In [30]:
#import required packages
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

rmse_val = [] #to store rmse values for different k

for K in range(20):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)
    
    model.fit(X_train, y_train)  #fit the model
    pred=model.predict(X_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

RMSE value for k=  1 is: 1.4276521876927784
RMSE value for k=  2 is: 1.4313223149941943
RMSE value for k=  3 is: 1.4680527303620856


KeyboardInterrupt: 

### ROC 

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random